In [94]:
import pyodbc
import pandas as pd
import os
import sqlalchemy as sa
import urllib

In [84]:
root = os.path.abspath(r"C:\Users\denni\temp_school\data")
parts_path = os.path.join(root,"parts.tsv")
business_path = os.path.join(root,"business.tsv")
person_path = os.path.join(root,"person.tsv")
repairs_path = os.path.join(root,"repairs.tsv")
users_path = os.path.join(root,"users.tsv")
vehicles_path = os.path.join(root,"vehicles.tsv")

## Utility Functions 

In [85]:
def compose_pyodbc_connection():
    SERVER = os.getenv("DB_SERVER", "MAORYZEN7\SQLEXPRESS")
    connection_string = 'Driver={SQL Server};Server=%s;Database=CS6400;Trusted_Connection=yes;' % ( SERVER )
    
    if os.getenv("PYODBC_AUTH")=="True":
        usr = os.getenv("PYODBC_USER")
        pw = os.getenv("PYODBC_PW")
        connection_string+='uid=%s;pwd=%s;'%(usr,pw)

    return connection_string


def insert_row(query:str,row):
    '''
    :param query:
    :return:
    '''
    connection_str = compose_pyodbc_connection()
    conn = pyodbc.connect(connection_str)

    cursor = conn.cursor()
    cursor.execute(query,row)
    conn.commit()
    cursor.close()

    return

def gen_query_add_row(table_name:str,row:tuple)->str:
    colQuery = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{table_name}';"

    colnames,_ = run_query(colQuery)

    colnames = ','.join([col[0] for col in colnames])
    row_len = len(row)

    row =",".join(["?" for i in range(row_len)])
    query = f"INSERT INTO {table_name}({colnames}) VALUES ({row}) "
    return query


def load_cleaned_data_to_db(table_name:str, df:pd.DataFrame):
    try: 
        for index, row in df.iterrows():
            query = gen_query_add_row(table_name,row)
            insert_row(query,row)
    except: 
        raise 

###  Current Schema to Provided Data Mapping 
**Our Schema_______________Provided Data**
```
Vehicle       -----   vehicle ('VIN', 'year', 'manufacturer_name', 
                               'model', 'description', 'invoice_price', 
                               'added_by', 'date_added', 'vehicle_type') 
Car           -----   vehicle (VIN,roof_type,back_seat_count) 
Convertible   -----   vehicle (VIN,roof_type, ) 
SUV           -----   vehicle (VIN,drive_train_type, num_cupholders)   
Truck         -----   vehicle (VIN,capacity, cover_type, num_rear_axles)   
VanMinivan    -----   vehicle (VIN,driver_side_door) 

Color         -----   vehicle (VIN,Colors)  
Manufacturer  -----   vehicle (manufacturer_name)  
Sale          -----   vehicle (VIN,'sold_by', 'sale_date', 'sold_price', 'customer')

Customer      -----   business/persons 
Business      -----   business
Person        -----   person

EmployeeUser  -----   users
Part          -----   parts
Repair        -----   repairs

```

# TODOs: 

1. Review provided data (content, types, cols) 
2. Breakdown input data to match our current schema
3. Correct / Map col names to match schema for each table
4. Correct datatypes to match schema 
5. Load Corrected tables to Database 

# Vehicles Data

<hr style="border:1px solid gray"> </hr>
**Our Schema_______________Provided Data**

```
Vehicle       -----   vehicle ('VIN', 'year', 'manufacturer_name', 
                               'model', 'description', 'invoice_price', 
                               'added_by', 'date_added', 'vehicle_type') 
Car           -----   vehicle (VIN,roof_type,back_seat_count) 
Convertible   -----   vehicle (VIN,roof_type, ) 
SUV           -----   vehicle (VIN,drive_train_type, num_cupholders)   
Truck         -----   vehicle (VIN,capacity, cover_type, num_rear_axles)   
VanMinivan    -----   vehicle (VIN,driver_side_door) 

Color         -----   vehicle (VIN,Colors)  
Manufacturer  -----   vehicle (manufacturer_name)  
Sale          -----   vehicle (VIN,'sold_by', 'sale_date', 'sold_price', 'customer')
```

### 01 Vehicle - Review provided data (content, types, cols)

In [86]:
vehicles = pd.read_csv(vehicles_path, sep = "\t")
vehicles.dtypes

VIN                   object
year                   int64
manufacturer_name     object
model                 object
Colors                object
description           object
invoice_price        float64
added_by              object
date_added            object
vehicle_type          object
number_doors         float64
back_seat_count      float64
roof_type             object
num_cupholders       float64
drive_train_type      object
num_rear_axles       float64
cover_type            object
capacity             float64
driver_side_door     float64
sold_by               object
sale_date             object
sold_price           float64
customer              object
dtype: object

In [87]:
vehicles.head()

,VIN,year,manufacturer_name,model,Colors,description,invoice_price,added_by,date_added,vehicle_type,...,num_cupholders,drive_train_type,num_rear_axles,cover_type,capacity,driver_side_door,sold_by,sale_date,sold_price,customer
0,00AIVKIDO01487633,2006,Lincoln,LS,Claret,NaN,5315.01,user09,7/17/2020,Van,...,NaN,NaN,NaN,NaN,NaN,1.0,user19,12/13/2020,6307.12,C0199518923
1,01WR7E2ZMCD216522,2006,Mitsubishi,FE145,Green,automatic safety features;,2572.00,user18,2/9/2021,SUV,...,1.0,RWD,NaN,NaN,NaN,NaN,user10,6/28/2021,3761.09,A9383285395
2,06XZOKN8UAY309163,2004,SAAB,9-5,Metallic,NaN,1177.06,user25,2/13/2020,SUV,...,3.0,FWD,NaN,NaN,NaN,NaN,roland,3/7/2020,1263.15,B7932999712
3,073HOEWCHAF741925,2013,INFINITI,QX56,"Black,Cream,Red",NaN,4488.90,user18,2/20/2021,Car,...,NaN,NaN,NaN,NaN,NaN,NaN,user10,3/6/2021,6395.17,C6390913308
4,07TSE0YYIZF380247,2003,Honda,CR125R,White,good gear; disc brake; disc brake;,2262.77,user20,4/11/2020,Van,...,NaN,NaN,NaN,NaN,NaN,1.0,user04,10/27/2020,2271.02,D8356291718


# Users Data

<hr style="border:1px solid gray"> </hr>

**Our Schema_______________Provided Data**

```
EmployeeUser  -----   users
```

### 01 Users - Review provided data (content, types, cols)

In [88]:
users= pd.read_csv(users_path, sep = "\t")
users.dtypes

username    object
password    object
u_f_name    object
u_l_name    object
roles       object
dtype: object

### 02 Users - Breakdown input data to match schema

In [89]:
# users data maps directly to EmployeeUser
# No additional work here

### 03 Users - Correct col names to match schema for each table

In [90]:
users.rename(columns={"username": "Username", 
                      "password": "Password",
                      "u_f_name": "First_name",
                      "u_l_name": "Last_name",
                      "roles": "Job_type"}, inplace=True)
users.dtypes

Username      object
Password      object
First_name    object
Last_name     object
Job_type      object
dtype: object

### 04 Users - Correct datatypes to match schema

In [91]:
#TODO
users = users.astype({"Username": "str", 
                       "Password": "str",
                       "First_name": "str",
                       "Last_name": "str",
                       "Job_type": "str"})
users.dtypes

Username      object
Password      object
First_name    object
Last_name     object
Job_type      object
dtype: object

In [92]:
users

,Username,Password,First_name,Last_name,Job_type
0,roland,roland,Roland,Around,"inventory_clerk,manager,owner,sales_person,ser..."
1,user01,pass01,Kris,Maisto,sales_person
2,user02,pass02,Dyan,Weglarz,manager
3,user03,pass03,Vi,Shields,inventory_clerk
4,user04,pass04,Pete,Butzen,sales_person
5,user05,pass05,Vi,Hoopengardner,inventory_clerk
6,user06,pass06,Sylvia,Rodenberger,sales_person
7,user07,pass07,Elke,Perez,inventory_clerk
8,user08,pass08,France,Matuszak,sales_person
9,user09,pass09,Novella,Gillian,inventory_clerk


### 05 Users - Load Data to database

In [102]:
#TODO 
def load_pandas_to_database(df, database_table_name):
    SERVER = os.getenv("DB_SERVER", "MAORYZEN7\SQLEXPRESS")
    connection_string = 'Driver={SQL Server};Server=%s;Database=CS6400;Trusted_Connection=yes;' % ( SERVER )
    connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"
    engine = sa.create_engine(connection_uri, fast_executemany=True)
    df.to_sql(database_table_name, engine, if_exists="append", index=False,chunksize = 10)

load_pandas_to_database(users,"database_table_name")


MemoryError: 

# Repairs Data

<hr style="border:1px solid gray"> </hr>

**Our Schema_______________Provided Data**

```
Repair        -----   repairs
```

### 01 Repairs - Review provided data (content, types, cols)

In [ ]:
repairs= pd.read_csv(repairs_path, sep = "\t")
repairs.dtypes

### 02 Repairs - Breakdown input data to match schema

In [ ]:
#TODO 

### 03 Repairs - Correct col names to match schema for each table

In [ ]:
#TODO 

### 04 Repairs - Correct datatypes to match schema

In [ ]:
#TODO 

### 05 Repairs - Correct datatypes to match schema

In [ ]:
#TODO 

# Parts Data

<hr style="border:1px solid gray"> </hr>

**Our Schema_______________Provided Data**

```
Part        -----   parts
```

### 01 Parts - Review provided data (content, types, cols)

In [ ]:
parts= pd.read_csv(parts_path, sep = "\t")
parts.dtypes 

### 02 Parts - Breakdown input data to match schema

In [ ]:
#TODO 

### 03 Parts - Correct col names to match schema for each table

In [ ]:
#TODO 

### 04 Parts - Correct datatypes to match schema

In [ ]:
#TODO 

### 05 Parts - Correct datatypes to match schema

In [ ]:
#TODO 

# Person Data

<hr style="border:1px solid gray"> </hr>

**Our Schema_______________Provided Data**

```
Customer      -----   business/persons
Person        -----   person
```

### 01 Person - Review provided data (content, types, cols)

In [ ]:
person= pd.read_csv(person_path, sep = "\t")
person.dtypes

### 02 Person - Breakdown input data to match schema

In [ ]:
#TODO 

### 03 Person - Correct col names to match schema for each table

In [ ]:
#TODO 

### 04 Person - Correct datatypes to match schema

In [ ]:
#TODO 

### 05 Person - Correct datatypes to match schema

In [ ]:
#TODO 

# Business Data

<hr style="border:1px solid gray"> </hr>

**Our Schema_______________Provided Data**

```
Customer      -----   business/persons
Business        -----   business
```

### 01 Business - Review provided data (content, types, cols)

In [ ]:
business= pd.read_csv(business_path, sep = "\t")
business.dtypes

### 02 Business - Breakdown input data to match schema

In [ ]:
#TODO 

### 03 Business - Correct col names to match schema for each table

In [ ]:
#TODO 

### 04 Business - Correct datatypes to match schema

In [ ]:
#TODO 

### 05 Business - Correct datatypes to match schema

In [ ]:
#TODO 